In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from scipy import signal

from lib_geos import fftvec

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = 10, 8
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['figure.titlesize'] = 8

# Python adaptation of lab_dispersion by Carl Tape for UAF GEOS 626
# Coding by Amanda McPherson (Jan 2021)

# Calls fftvec.py

# Run this cell at the beginning of each session

In [ ]:
# Input parameters, constants, and data

deg = 180/np.pi

# target periods for measurements
Ttarvec = np.array([20, 30, 40, 50])
ftarvec = 1/Ttarvec
numtar = len(ftarvec)

# distance between PAS and NEE (km)
delx = 331

# axes limits for dispersion plots of speed (km/s) vs period (s)
ax_1 = [18, 52, 2.8, 4.6]
    
# we are told that the phase velocity at each period must fall within these ranges
cran = np.array([[3.1, 3.9], [3.0, 4.3], [3.3, 4.5], [3.3, 4.5]])

# load data files (FUTURE: we should fetch these from IRIS)
file1 = './data/pas.dat'
file2 = './data/nee.dat'
ti, ynee = np.loadtxt(file2,unpack=True)
ti_pas, ypas = np.loadtxt(file1,unpack=True)

# time step
dt = ti[1] - ti[0]

# needed for butter filter
fs = 1/dt

# for FFT an even number is helpful, so chop off the first point
mask = np.ones(len(ti), dtype=bool)
mask[0] = False
ti = ti[mask]
ynee = ynee[mask]
ypas = ypas[mask]
nt = len(ti)

In [ ]:
# define a bandpass function
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html
def bandpass(y,fa,fb):
    b, a = signal.butter(4, [fa, fb], btype='bandpass', fs=fs)
    ybp = signal.filtfilt(b,a,y)
    return ybp

In [ ]:
# Plotting seismograms

# use hilbert transform to compute envelope
ypasen = abs(signal.hilbert(ypas))
yneeen = abs(signal.hilbert(ynee))

xran = np.array([ti[0], ti[-1]])

# plotting
fig1, ax1 = plt.subplots(2,1,num=1)

ax1[0].plot(ti,ypas,'b')
#ax1[0].plot(ti,ypasen,'k--',ti,-ypasen,'k--')   # envelope
ax1[0].set(xlabel='Time (s)', ylabel='Amplitude', title='Pasadena, LHZ')
ax1[0].set_xlim(xran)

ax1[1].plot(ti,ynee,'r')
#ax1[1].plot(ti,yneeen,'k--',ti,-yneeen,'k--')   # envelope
ax1[1].set(xlabel='Time (s)', ylabel='Amplitude', title='Needles, LHZ')
ax1[1].set_xlim(xran)

plt.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
# COMPUTE FOURIER TRANSFORM, THEN PLOT AMPLITUDE SPECTRUM

# frequency vector
npt = nt
f = fftvec(ti)          # note: negative frequencies

# p = PAS/Pasadena
Hp = np.fft.fft(ypas)
Ap = abs(Hp)            # =sqrt(H.*conj(H)), where P=H.*conj(H) is the power spectral density
# n = NEE/Needles
Hn = np.fft.fft(ynee)
An = abs(Hn)

In [ ]:
# explore these to see various details with the FFT (change False to True)
if False:
    y = ypas
    H = Hp
    A = Ap
    imax = int(npt/2+1)
    ip = np.arange(1,imax+1)
    
    print('check the first entry of the FFT:')
    print('sum(ypas) = \n', np.sum(ypas))
    print('mean(ypas)*npt = \n', np.mean(ypas)*npt)
    print('H[0] = \n', H[0])
    
    # check the difference between abs and power
    # if z = a + bi, then abs(z) = sqrt(z z*)
    print(np.linalg.norm( A*A - H*np.conjugate(H) ) / np.linalg.norm( A ))
    
    # compare IFFT[FFT[y(t)]] with y(t)
    plt.figure()
    plt.plot(ti,y,'b')
    plt.plot(ti,np.fft.ifft(H),'r--')
    
    # check the ordering of the complex entries of H
    plt.figure()
    plt.plot(np.real(H[1:imax-1]) - np.real(H[npt:imax-1:-1]),'.')
    plt.axis([0, 500, -1, 1])
    
    plt.show()

In [ ]:
# plot the spectra for PAS and NEE

# positive and negative frequencies
plt.figure()
plt.plot(f,Ap,'b',lw=0.5,label='PAS')
plt.plot(f,An,'r',lw=0.5,label='NEE')
plt.legend(loc='upper right')
plt.xlabel('frequency (Hz)')
plt.ylabel('spectral amplitude')

# positive frequencies only
ipos = np.nonzero(f>0)

plt.figure()
plt.plot(f[ipos],Ap[ipos],'b',lw=0.5,label='PAS')
plt.plot(f[ipos],An[ipos],'r',lw=0.5,label='NEE')
plt.legend(loc='upper right')
plt.xlabel('frequency (Hz)')
plt.ylabel('spectral amplitude')

plt.show()

In [ ]:
# experiment with bandpass (change False to True)
if False:
    # create random signal
    sig = np.random.randn(500)*4 + np.random.randn(500).cumsum()
    # filter it
    sig_filt = bandpass(sig,0.1,0.25)
    
    # Plot the filtered and unfiltered signals
    plt.figure()
    plt.plot(sig,'b')
    plt.plot(sig_filt,'r')
    plt.show()

In [ ]:
# CODE HERE FOR GROUP SPEED (use a bandpass filter)



In [ ]:
# SETUP FOR HARMONICS

# fourier transform of Pasadena and Needles seismograms
Hp = np.fft.fft(ypas)
Hn = np.fft.fft(ynee)

# Check out the dimensions of variables
print('dimensions of variables:')
print('ti   :',ti.shape)
print('ypas :',ypas.shape)
print('ynee :',ynee.shape)
print('f    :',f.shape)
print('Hp   :',Hp.shape)
print('Hn   :',Hn.shape)

In [ ]:
# CODE HERE FOR HARMONICS (No bandpass needed)

tlims = [2580, 2780]

for ii in range(numtar):
    # target frequency for harmonic
    ftar = ftarvec[ii]
    
    # initialize harmonics
    Hp2 = np.array(np.zeros(npt), dtype=complex)
    Hn2 = np.array(np.zeros(npt), dtype=complex)
    
    # get the indices for ftar and -ftar from the frequency vector
    # (this will avoid having python tell you that it will ignore the
    # complex conjugate parts when using ifft)
    itemp = np.argsort(abs(abs(f)-ftar))
    itemp = itemp[0:2]
    print('ii = ',ii,', itemp = ',itemp,', f[itemp] =',f[itemp])
    
    # CODE HERE FOR HARMONICS
    #Hp2 
    #Hn2 
    

In [ ]:
# CODE HERE FOR PHASE SPEED

